# Analyse de la distribution des élus par catégorie socio-professionnelle en France


Le présent notebook vise à analyser la distribution des élus en France selon leur catégorie socio-professionnelle. Les données utilisées proviennent du Répertoire National des Élus, un fichier ouvert accessible sur data.gouv.fr (https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv), qui contient des informations sur les maires et autres élus locaux en France.

Ces données permettent de mieux comprendre les caractéristiques socio-démographiques des élus, notamment leur catégorie socio-professionnelle. L'analyse se concentre sur la répartition de ces élus selon leur CSP, afin de mettre en lumière les tendances et les disparités éventuelles.


## Méthodologie

L'analyse a été réalisée en utilisant le langage SQL pour agréger les données et le langage Python avec les bibliothèques DuckDB et Plotly pour la visualisation. La requête SQL a permis de regrouper les élus par catégorie socio-professionnelle et de compter le nombre d'élus dans chaque catégorie. Les résultats ont ensuite été triés par ordre décroissant pour identifier les CSP les plus représentées.


## Analyse

La requête SQL utilisée pour l'analyse est la suivante :
La requête regroupe les élus par "Code de la catégorie socio-professionnelle" et "Libellé de la catégorie socio-professionnelle", et compte le nombre d'élus dans chaque groupe.

Les données ont été ensuite utilisées pour créer un diagramme en barres horizontales représentant la distribution des élus par CSP.


## Visualisation

La visualisation est un diagramme en barres horizontales créé avec Plotly, représentant la distribution des élus par catégorie socio-professionnelle. Les CSP sont triées par ordre croissant du nombre d'élus, et le graphique est configuré pour afficher des informations détaillées au survol de chaque barre.

 
## Interprétation

Les résultats de l'analyse offrent une vue d'ensemble de la distribution des élus locaux en France selon leur catégorie socio-professionnelle, permettant d'identifier les tendances et les disparités dans la représentation des différentes CSP parmi les élus.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://static.data.gouv.fr/resources/repertoire-national-des-elus-1/20250613-142903/elus-maires-mai.csv", safe_mode=True)
print("✅ Données chargées avec succès")

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" SELECT 
  "Code de la catégorie socio-professionnelle" AS csp_code,
  "Libellé de la catégorie socio-professionnelle" AS csp_libelle,
  COUNT(*) AS nb_elus
FROM 
  loaded_dataset
GROUP BY 
  "Code de la catégorie socio-professionnelle", 
  "Libellé de la catégorie socio-professionnelle"
ORDER BY 
  nb_elus DESC """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 72 lignes


,csp_code,csp_libelle,nb_elus
0,74,Ancien cadre,5555
1,12,Agriculteur sur moyenne exploitation,2858
2,77,Ancien employé,2228
3,71,Ancien agriculteur exploitant,1941
4,37,Cadre administratif et commercial d'entreprise,1901


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui permet de créer des visualisations interactives et personnalisées. Cette technologie est adaptée pour représenter des données complexes de manière claire et concise, notamment pour des diagrammes en barres horizontales comme celui-ci. Plotly Express offre une grande flexibilité dans la mise en forme et l'interactivité, rendant la visualisation plus engageante et informative.

In [4]:
import pandas as pd
import duckdb as ddb

import plotly.express as px
import pandas as pd

# Trie des CSP par nombre d'élus décroissant
df_sorted = df.sort_values('nb_elus', ascending=True)

# Création du diagramme en barres horizontales
dataviz = px.bar(
    df_sorted,
    y='csp_libelle',
    x='nb_elus',
    orientation='h',
    title='Distribution des élus par catégorie socio-professionnelle',
    labels={'csp_libelle': 'Catégorie socio-professionnelle', 'nb_elus': 'Nombre d\'élus'},
    color_discrete_sequence=['#1f77b4'],
    height=1200
)

# Mise en forme
dataviz.update_layout(
    margin=dict(l=20, r=20, t=40, b=20),
    font=dict(family='Segoe UI', size=12),
    xaxis=dict(showgrid=True, gridcolor='LightGrey'),
    yaxis=dict(showgrid=False, tickfont=dict(size=10)),
    plot_bgcolor='white'
)

# Infobulle plus détaillée
dataviz.update_traces(
    hovertemplate='<b>%{y}</b><br>Élus: %{x:,}<extra></extra>'
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250802_162551.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250802_162551.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250802_162551.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250802_162551.png


--> Image Plotly sauvegardée avec succès.
